In [51]:
# Libraries

import pandas as pd
import random
import ezodf

In [52]:
# Gets reelset, symbol and payment table information from Excel file.

Symbols = []
PayTable = []
ReelSets = []

# Excel File Path
file_name = 'atkins_diet_parsheet.ods'

# Reading data from excel file
data_symbols = pd.read_excel(file_name, sheet_name = 'Symbols')
data_paytable = pd.read_excel(file_name, sheet_name = 'Pay Table')
data_reelset = pd.read_excel(file_name, sheet_name = 'Reelset')
data_lines = pd.read_excel(file_name, sheet_name = 'Line Shape')

Symbols = data_symbols.iloc[2:14, 2].tolist()
PayTable_Symbols = [
    [int(x) if isinstance(x, (float, int)) else x for x in data_paytable.iloc[:, i].dropna().tolist()[1:]]
    for i in range(5, 0, -1)
]
PayTable = [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]] + PayTable_Symbols
ReelSet = [data_reelset.iloc[:, i].dropna().tolist() for i in range(1, 6)]
Lines = [data_lines.iloc[i, 33:38].tolist() for i in range(2, 22)]

Free_Spin_Counts = {3: 10, 4: 10, 5: 10}
Free_Spin_Wins_Multiplier = 3

In [53]:
class Spin:
    def __init__(self, ReelSet, bet):
        self.ReelSet = ReelSet
        self.bet = bet
        self.spin_result = []
        self.total_bet = 0  

    def execute(self, bet):
        """ Choose a random index for each reel and create a sequential spin. """
        self.spin_result = []
        for reel in self.ReelSet:
            # Choose a random index
            random_index = random.randint(0, len(reel) - 1)
            
            # Edit index limits with mod
            left_index = (random_index - 1) % len(reel)
            middle_index = random_index
            right_index = (random_index + 1) % len(reel)
            
            # Generate sequential spin results
            result_column = [
                reel[left_index],
                reel[middle_index],
                reel[right_index]
            ]
            self.spin_result.append(result_column)

        self.total_bet += bet

        return self.spin_result
    

In [54]:
# Create Spin and Win objects
spin_machine = Spin(ReelSet, bet = len(Lines))

# Kullanicidan spin sayisini al
spin_count = int(input("How many spins do you want to create? "))
print(spin_count)

10000000


In [55]:
class Win:
    def __init__(self, paytable, symbols, wild_symbol="Wild", scatter_symbol = "Scat"):
        
        self.paytable = paytable
        self.symbols = symbols
        self.wild_symbol = wild_symbol
        self.scatter_symbol = scatter_symbol
        self.total_win = 0  

        self.basegame_line_win = 0
        self.basegame_scat_win = 0
        self.freespin_line_win = 0
        self.freespin_scat_win = 0

        self.total_basegame_line_win = 0  
        self.total_basegame_scatter_win = 0
        self.total_freespin_line_win = 0
        self.total_freespin_scat_win = 0
        self.free_spin_count = 0  

        self.payout_distribution = {symbol: [0, 0, 0, 0, 0] for symbol in symbols}  # 1-5 kazançlar

        self.total_spins = 0
        self.total_scatter_triggers = 0
        self.total_freespins_given = 0

    def line_win_check(self, spin_result, Lines, mode="basegame"):
        self.total_spins += 1
        total_line_win = 0 

        for i in range(len(Lines)):
            line_symbols = [spin_result[j][Lines[i][j]] for j in range(5)]  # Get the symbols on the reels
            line_win_symbol = line_symbols[0]  # Base on first symbol
            same_symbol_count = 1

            if line_win_symbol == "Scat":  # If there is "Scat", there is no payout
                continue  

            for symbol in line_symbols[1:]:
                if symbol == line_win_symbol or symbol == self.wild_symbol or line_win_symbol == self.wild_symbol:
                    same_symbol_count += 1
                    if line_win_symbol == self.wild_symbol and symbol != self.wild_symbol:
                        line_win_symbol = symbol
                else:
                    break

            # Calculate payout if wild symbol can be found in "line_win_symbol"
            symbol_index = self.symbols.index(line_win_symbol) if line_win_symbol in self.symbols else -1
            basegame_line_win = 0
            
            if symbol_index != -1 and same_symbol_count < len(self.paytable):
                basegame_line_win = self.paytable[same_symbol_count][symbol_index]

                # Add wins according to mode
                if mode == "basegame":
                    self.total_basegame_line_win += basegame_line_win
                    self.payout_distribution[line_win_symbol][same_symbol_count - 1] += basegame_line_win
                elif mode == "freespin":
                    self.total_freespin_line_win += basegame_line_win * Free_Spin_Wins_Multiplier

            total_line_win += basegame_line_win  # Adds up the winnings of all lines

        return total_line_win  # Return total winnings

    
    def scatter_win_check(self, spin_result, bet, mode = "basegame"):   

        all_symbols = [symbol for reel in spin_result for symbol in reel]
        scatter_count = all_symbols.count(self.scatter_symbol)
        scatter_index = self.symbols.index(self.scatter_symbol) if self.scatter_symbol in self.symbols else -1

        basegame_scat_win = 0
        if scatter_index != -1 and scatter_count < len(self.paytable):
            basegame_scat_win = self.paytable[scatter_count][scatter_index]*bet

            # Add to total and distribution only in "basegame" mode
            if mode == "basegame":
                self.total_basegame_scatter_win += basegame_scat_win

            # If in "freespin" mode, add only freespin winnings
            elif mode == "freespin":
                self.total_freespin_scat_win += basegame_scat_win * Free_Spin_Wins_Multiplier

        return basegame_scat_win
    
    def freespin_win_check(self, spin_result, bet):
        all_symbols = [symbol for reel in spin_result for symbol in reel]
        scatter_count = all_symbols.count(self.scatter_symbol)

        if scatter_count in Free_Spin_Counts:
            # Increase scatter trigger count
            self.total_scatter_triggers += 1

            total_freespin_win = 0
            total_fs_scat_win = 0

            # Increase the number of triggered free spins
            free_spins = Free_Spin_Counts[scatter_count]
            self.total_freespins_given += free_spins  # Increase the total number of free spins awarded

            freespin_line_win = 0
            total_fs_line_win = 0
            current_spin = 0  

            while free_spins > 0:  # Continue until the free spins count reaches zero
                current_spin += 1
                spin_result = spin_machine.execute(bet=0)

                freespin_line_win = self.line_win_check(spin_result, Lines, mode="freespin")
                total_fs_line_win += freespin_line_win

                # New scatter control
                freespin_scat_win = 0
                new_scatter_count = [symbol for reel in spin_result for symbol in reel].count(self.scatter_symbol)

                if new_scatter_count in Free_Spin_Counts:
                    additional_spins = Free_Spin_Counts[new_scatter_count]
                    free_spins += additional_spins

                    # Add new freespin trigger to counters
                    self.total_scatter_triggers += 1  # New trigger
                    self.total_freespins_given += additional_spins  # Number of free spins added

                    freespin_scat_win = self.scatter_win_check(spin_result, bet, mode="freespin")
                    total_fs_scat_win += freespin_scat_win

                # One spin completed
                free_spins -= 1

            total_fs_line_win = total_fs_line_win * Free_Spin_Wins_Multiplier
            total_fs_scat_win = total_fs_scat_win * Free_Spin_Wins_Multiplier

            total_freespin_win = total_fs_line_win + total_fs_scat_win

            return total_fs_line_win, total_fs_scat_win, total_freespin_win
        
        else:
        # If scatter_count is not in Free_Spin_Counts
            return 0, 0, 0  


In [56]:
win_checker = Win(PayTable, Symbols)
bet = len(Lines)
spin_win_distribution = {}

# Run spins and calculate Line RTP
for spin_index in range(spin_count):
    spin_result = spin_machine.execute(bet)
    line_win = win_checker.line_win_check(spin_result, Lines)
    scatter_win = win_checker.scatter_win_check(spin_result, bet)
    fs_line_win, fs_scat_win, freespin_win = win_checker.freespin_win_check(spin_result, bet)

    line_win = line_win if line_win is not None else 0
    scatter_win = scatter_win if scatter_win is not None else 0
    fs_line_win = fs_line_win if fs_line_win is not None else 0
    fs_scat_win = fs_scat_win if fs_scat_win is not None else 0

    # Calculate total earnings
    total_spin_win = line_win + scatter_win + fs_line_win + fs_scat_win

    # Add the winning amount to the distribution dictionary
    if total_spin_win not in spin_win_distribution:
        spin_win_distribution[total_spin_win] = 0
    spin_win_distribution[total_spin_win] += 1  # This gain occurred once again
    
total_win = (win_checker.total_basegame_line_win + win_checker.total_basegame_scatter_win + win_checker.total_freespin_line_win + win_checker.total_freespin_scat_win)

# Base Game Line RTP results
print("Total Bet:", spin_machine.total_bet, "\n")
print("Base Game Line Win:", win_checker.total_basegame_line_win )
print(f"Base Game Line RTP: {(win_checker.total_basegame_line_win)/spin_machine.total_bet*100:.2f}%", "\n")

# Base Game Scatter RTP results
print("Base Game Scatter Win:", win_checker.total_basegame_scatter_win)
print(f"Base Game Scatter RTP: {win_checker.total_basegame_scatter_win/spin_machine.total_bet*100:.2f}%", "\n")

# Freespin RTP results
print("FreeSpin Line Win:", win_checker.total_freespin_line_win)
print("FreeSpin Scatter Win:", win_checker.total_freespin_scat_win)
freespin_line_rtp = win_checker.total_freespin_line_win/spin_machine.total_bet*100
freespin_scat_rtp = win_checker.total_freespin_scat_win/spin_machine.total_bet*100
print(f"Bonus RTP:{freespin_line_rtp + freespin_scat_rtp:.2f}%", "\n")

# Total RTP result
print("Total Win:", total_win)
print(f"Final RTP: {total_win/spin_machine.total_bet*100:.2f}%")


Total Bet: 200000000 

Base Game Line Win: 126876527
Base Game Line RTP: 63.44% 

Base Game Scatter Win: 13948300.0
Base Game Scatter RTP: 6.97% 

FreeSpin Line Win: 47765568
FreeSpin Scatter Win: 5333100
Bonus RTP:26.55% 

Total Win: 193923495.0
Final RTP: 96.96%


In [ ]:
# Function for save the spin distribution

def save_spin_distribution_to_ods(spin_win_distribution, filename="spin_win_distribution.ods"):
    """Saves the spin win distribution to an ODS file."""
    # Create a new .ods document
    spreadsheet = ezodf.newdoc(doctype="ods", filename=filename)

    # Add a new sheet
    sheet = ezodf.Sheet("Spin Win Distribution", size=(len(spin_win_distribution) + 1, 2))
    spreadsheet.sheets += sheet

    # Add headers
    sheet[0, 0].set_value("Win Amount")
    sheet[0, 1].set_value("Spin Count")

    # Sort the spin_win_distribution by win_amount (key)
    sorted_spin_win_distribution = sorted(spin_win_distribution.items(), key=lambda x: x[0])

    # Populate the sheet with data
    for row_index, (win_amount, spin_count) in enumerate(sorted_spin_win_distribution, start=1):
        sheet[row_index, 0].set_value(win_amount)
        sheet[row_index, 1].set_value(spin_count)

    # Save the .ods file
    spreadsheet.save()
    print(f"Data successfully saved to {filename}!")

save_spin_distribution_to_ods(spin_win_distribution)

Data successfully saved to spin_win_distribution.ods!
